# Primary decompositions from paper

In [1]:
# imports
import numpy
import sympy

from syngular import Field, Ring, QRing, Ideal, TemporarySetting
from lips import Particles
from lips.algebraic_geometry.tools import lips_covariant_symbols
from lips.algebraic_geometry.covariant_ideal import LipsIdeal

## Build the ring

In [2]:
multiplicity = m = 5
oPs = Particles(multiplicity)
oPs.make_analytical_d()

In [3]:
# 𝟑, 𝟒, 𝟓 @ 2+3 point
abX, bbX, cbX, dbX = sympy.symbols(f"ab{m-2}, bb{m-2}, cb{m-2}, db{m-2}")
abY, bbY, cbY, dbY = sympy.symbols(f"ab{m-1}, bb{m-1}, cb{m-1}, db{m-1}")
abZ, bbZ, cbZ, dbZ = sympy.symbols(f"ab{m}, bb{m}, cb{m}, db{m}")
mt = sympy.symbols("mt")

In [4]:
oPs[m-2]._r2_sp = numpy.array([[abX, bbX], [cbX, dbX]])
oPs[m-2]._r2_sp_to_r2_sp_b()
oPs[m-1]._r2_sp = numpy.array([[abY, bbY], [cbY, dbY]])
oPs[m-1]._r2_sp_to_r2_sp_b()
oPs[m].r2_sp = numpy.array([[abZ, bbZ], [cbZ, dbZ]])  # leg 5 won't be used (for efficiency reasons)
oPs[m]._r2_sp_to_r2_sp_b()

In [5]:
# see S_{scalar-tops}, eq. 2.24 of paper
S5 = Ring('0', lips_covariant_symbols(m - 3) + (abX, bbX, cbX, dbX, abY, bbY, cbY, dbY, ), 'dp')  # abZ, bbZ, cbZ, dbZ - explicitly remove 5 from ring
S5

0, (a1, b1, c1, d1, a2, b2, c2, d2, ab3, bb3, cb3, db3, ab4, bb4, cb4, db4), dp

In [6]:
JLambda5 = Ideal(S5, [oPs(f"s_{m-2}-s_{m-1}"), ])  #  + oPs.total_mom.flatten().tolist() - momentum conservation has been pre-solved
JLambda5

Ideal over Ring
    0, (a1, b1, c1, d1, a2, b2, c2, d2, ab3, bb3, cb3, db3, ab4, bb4, cb4, db4), dp
generated by
    ab3*db3 - ab4*db4 - bb3*cb3 + bb4*cb4

In [7]:
# see R_{scalar-tops}, eq. 2.25 of paper
R5 = QRing(JLambda5.ring, JLambda5)
R5

0, (a1, b1, c1, d1, a2, b2, c2, d2, ab3, bb3, cb3, db3, ab4, bb4, cb4, db4), dp;
ideal i_ = ab3*db3 - ab4*db4 - bb3*cb3 + bb4*cb4;
qring q = std(i_)

### Build a $\mathbb{F}_p$ phase space point for further numerical checks

In [8]:
oPsFF = Particles(8, field=Field("finite field", 2 ** 31 - 19, 1), seed=0)
oPsFF._singular_variety(("⟨34⟩+[34]", "⟨34⟩-⟨56⟩", "⟨56⟩+[56]"), (1, 1, 1), seed=0)
oPsFF.mt = - oPsFF("⟨34⟩")
oPsFF = oPsFF.cluster([[1, ], [2, ], [3, 4], [5, 6], [7, 8]], massive_fermions=((3, 'u', 1), (4, 'd', 1)))

## Primary decompositions

In [9]:
# for quickest results use seminumerical_dim_computation=True, nbr_points=0 in test_primality
# some checks also complete with the iterated_degbound_computation strategy, 
# and this may be quicker for simple checks, but it may not terminate for harder cases

### Decomposition of $\big\langle ⟨1|𝟑|𝟓|1⟩, [2|𝟓|𝟒|2] \big\rangle$

In [10]:
I = Ideal(R5, [sympy.expand(oPs("⟨1|𝟑|2+𝟑+𝟒|1⟩")),  # = ⟨1|𝟑|𝟓|1⟩
               sympy.expand(oPs("[2|1+𝟑+𝟒|𝟒|2]")),  # = [2|𝟓|𝟒|2]
              ])

In [11]:
J = Ideal(R5, [sympy.expand(oPs("⟨1|𝟑|2]")),
               sympy.expand(oPs("⟨1|𝟒|2]")),
               sympy.expand(oPs("⟨1|𝟑|2+𝟑+𝟒|1⟩")),  # = ⟨1|𝟑|𝟓|1⟩
               sympy.expand(oPs("[2|1+𝟑+𝟒|𝟒|2]")),  # = [2|𝟓|𝟒|2]
              ], )

In [12]:
K = Ideal(R5, [sympy.expand(oPs("⟨1|𝟑|2+𝟑+𝟒|1⟩")),  # = ⟨1|𝟑|𝟓|1⟩
               sympy.expand(oPs("[2|1+𝟑+𝟒|𝟒|2]")),  # = [2|𝟓|𝟒|2]
              ] +
          (numpy.vectorize(sympy.expand)(oPs("|1+𝟑|2]⟨1|2+𝟑+𝟒|-|1+𝟑+𝟒|2]⟨1|𝟑|"))).flatten().tolist()  # = |1+𝟑|2]⟨1|𝟓|-|𝟓|2]⟨1|𝟑|
         )

In [13]:
# prove primary decomposition eq. 2.31
assert J.test_primality(verbose=False, seminumerical_dim_computation=True, nbr_points=0)
assert K.test_primality(verbose=False, seminumerical_dim_computation=True, nbr_points=0)
assert I == J & K and J != K

In [14]:
# verify identity eq. 2.32
assert numpy.all(oPsFF("|𝟓|2]⟨1|𝟑|𝟓|1⟩[2|+|1⟩[2|𝟓|𝟒|2]⟨1|𝟓|") == oPsFF("⟨1|𝟓|2](|𝟓|2]⟨1|𝟑|-|1+𝟑|2]⟨1|𝟓|)"))

### Decomposition  of  $\big\langle ⟨7|3+4|5+6|7⟩, Δ_{712|34|56}\big\rangle$

In [15]:
# from arXiv:2203.17170, eq. 2.34 in this paper

In [16]:
I = LipsIdeal(7, ("⟨7|3+4|5+6|7⟩", "Δ_712|34|56"))
J = LipsIdeal(7, ("⟨7|3+4|5+6|7⟩", "Δ_712|34|56", "(|3+4|5+6|7⟩-|5+6|3+4|7⟩)(⟨7|5+6|3+4|-⟨7|3+4|5+6|)", ))

In [17]:
# this takes a bit of time, might want to skip it (one-day timeout is very over-generous)
with TemporarySetting("syngular", "TIMEOUT", 60 * 60 * 24):
    assert I == J

In [18]:
K = LipsIdeal(7, ("Δ_712|34|56", "(|3+4|5+6|7⟩-|5+6|3+4|7⟩)", ))

In [19]:
# this takes a bit of time, might want to skip it (one-day timeout is very over-generous)
with TemporarySetting("syngular", "TIMEOUT", 60 * 60 * 24):
    assert J / K == K

In [20]:
# this takes 5 min or so
assert K.test_primality(verbose=False, seminumerical_dim_computation=True, nbr_points=0, timeout_fpoly=5, timeout_dim=2)

### Decomposition  of  $\big\langle Δ_{2𝟒|1𝟑|𝟓}, [2|𝟓|𝟒|2] \big\rangle$

In [21]:
# eq. 2.35 of paper

In [22]:
I = Ideal(R5, [4 * sympy.expand(oPs("Δ_2𝟒|1𝟑|𝟓")),
               sympy.expand(oPs("[2|1+𝟑+𝟒|𝟒|2]")),  # [2|𝟓|𝟒|2]
              ])

In [23]:
assert I.test_primality(verbose=False, seminumerical_dim_computation=True, nbr_points=0, timeout_fpoly=5, timeout_dim=2, astuple=True) == (True, False)  # primary but not prime

Was irreducible at the chosen point. Continuing test.                    

In [24]:
J = Ideal(R5, [4 * sympy.expand(oPs("Δ_2𝟒|1𝟑|𝟓")),
               sympy.expand(oPs("[2|1+𝟑+𝟒|𝟒|2]"))] + 
               (numpy.vectorize(sympy.expand)(oPs("(|2+𝟒|1+2+𝟑+𝟒|2]-|1+2+𝟑+𝟒|2+𝟒|2])([2|1+2+𝟑+𝟒|2+𝟒|-[2|2+𝟒|1+2+𝟑+𝟒|)"))).flatten().tolist()
              )

In [25]:
assert I == J

In [26]:
rJ = Ideal(R5, [4 * sympy.expand(oPs("Δ_2𝟒|1𝟑|𝟓")), ] +
           (numpy.vectorize(sympy.expand)(oPs("(|2+𝟒|1+2+𝟑+𝟒|2]-|1+2+𝟑+𝟒|2+𝟒|2])"))).flatten().tolist()
          )

In [27]:
assert sympy.expand(oPs("[2|1+𝟑+𝟒|𝟒|2]")) in rJ

In [28]:
assert J / rJ == rJ

In [29]:
assert rJ.test_primality(verbose=False, seminumerical_dim_computation=True, nbr_points=0, timeout_fpoly=5, timeout_dim=2)

In [30]:
# eq. 2.36 - see also https://gdelaurentis.github.io/antares-results/antares_results/ttH/qqttH/pm/coeffs/tri13x24xm0m.html
from antares.terms.terms import Terms
Terms("""
+(7/3m_t(([2|𝟓|2+𝟒|-[2|2+𝟒|𝟓|)@(|2]⟨𝟑|1+𝟑|𝟒]+|𝟒]⟨𝟑|𝟓|2])))/([2|𝟒|𝟓|2]⟨1|𝟓|2])
+('21435', True, '+')
""")

Terms("""+(7/3m_t(([2|𝟓|2+𝟒|-[2|2+𝟒|𝟓|)@(|2]⟨𝟑|1+𝟑|𝟒]+|𝟒]⟨𝟑|𝟓|2])))/([2|𝟒|𝟓|2]⟨1|𝟓|2])
+('21435', True, '+')""")

### Decomposition  of  $\big\langle Δ_{12|3|45}, Δ_{12|3|4|5} \big\rangle$

In [31]:
# From here on bold font is assumed (and dropped) for 3, 4, 5

In [32]:
# eq. 2.40 - 2.41
assert oPsFF("s_12 Δ_12|3|4|5 - Δ_53|12|4 Δ_12|3|45 + (s12 tr(3|4) / 2 - tr(1+2|3) tr(1+2|4) / 4)^2")
assert oPsFF("s_3 Δ_12|3|4|5 - Δ_512|3|4 Δ_12|3|45 + (s_3 tr(1+2|4) / 2 - tr(3|4) tr(1+2|3) / 4)^2")

In [33]:
I = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|3|4|5")),
               sympy.expand(4 * oPs("Δ_12|3|45"))], )

In [34]:
J = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|3|4|5")),
               sympy.expand(4 * oPs("Δ_12|3|45")),
               sympy.expand(16 * oPs("(s12tr(3|4)/2-tr(1+2|3)tr(1+2|4)/4)^2")),
               sympy.expand(16 * oPs("(s_3tr(1+2|4)/2-tr(3|4)tr(1+2|3)/4)^2"))], )

In [35]:
rJ = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|3|45")),
                sympy.expand(4 * oPs("(s12tr(3|4)/2-tr(1+2|3)tr(1+2|4)/4)")),
                sympy.expand(4 * oPs("(s_3tr(1+2|4)/2-tr(3|4)tr(1+2|3)/4)"))], )

In [36]:
assert sympy.expand(4 * oPs("Δ_12|3|4|5")) in rJ
assert I == J
assert J / rJ == rJ

In [37]:
# this takes a bit of time
assert rJ.test_primality(verbose=False, seminumerical_dim_computation=True, nbr_points=0, timeout_fpoly=5)

Was irreducible at the chosen point. Continuing test.                    

### Decomposition  of  $\big\langle Δ_{12|3|4|5}, Δ_{12|34|5} \big\rangle$

In [38]:
# eq. 2.44
assert oPsFF("4Δ_12|3|4|5 - (s_34 - 4s_3) Δ_12|34|5 + s_34tr(1+2|3-4)^2/4") == 0

In [39]:
I = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|3|4|5")),
               sympy.expand(4 * oPs("Δ_12|34|5"))], )

In [40]:
J = Ideal(R5, [sympy.expand(4 * oPs("s_34")),
               sympy.expand(4 * oPs("tr(1+2|3+4)^2")),
              ], )

In [41]:
rJ = Ideal(R5, [sympy.expand(4 * oPs("s_34")),
                sympy.expand(4 * oPs("tr(1+2|3+4)")),
               ], )

In [42]:
K = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|34|5")),
               sympy.expand(4 * oPs("tr(1+2|3-4)^2")),
              ], )

In [43]:
rK = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|34|5")),
                sympy.expand(4 * oPs("tr(1+2|3-4)")),
               ], )

In [44]:
assert I == J & K

In [45]:
assert rJ.test_primality(verbose=False, seminumerical_dim_computation=True, timeout_fpoly=5, nbr_points=0)

In [46]:
assert rK.test_primality(verbose=False, seminumerical_dim_computation=True, timeout_fpoly=5, nbr_points=0)

Was irreducible at the chosen point. Continuing test.                    

### Decomposition  of  $\big\langle Δ_{12|3|45}, s_{123}-s_{3} \big\rangle$

In [47]:
# eq. 2.46
assert oPsFF("4Δ_12|3|45 - (s_123 - s_3) tr(1+2|3) + s_12 (tr(1+2|3) + 4s_3)") == 0

In [48]:
I = Ideal(R5, [sympy.expand(4 * oPs("Δ_12|3|45")),
               sympy.expand(4 * oPs("(s_123-s_3)"))], )

In [49]:
J = Ideal(R5, [sympy.expand(4 * oPs("(s_123-s_3)")),
                   sympy.expand(4 * oPs("(tr(1+2|3)+4s_3)"))], )

In [50]:
K = Ideal(R5, [sympy.expand(4 * oPs("(s_123-s_3)")),
               sympy.expand(4 * oPs("[1|2]"))], )

In [51]:
L = Ideal(R5, [sympy.expand(4 * oPs("(s_123-s_3)")),
               sympy.expand(4 * oPs("⟨1|2⟩"))], )

In [52]:
assert I == J & K & L and J != K and J != L and K != L

In [53]:
assert J.test_primality(verbose=False, seminumerical_dim_computation=True, timeout_fpoly=5, nbr_points=0)

In [54]:
assert K.test_primality(verbose=False, seminumerical_dim_computation=True, timeout_fpoly=5, nbr_points=0)

In [55]:
assert L.test_primality(verbose=False, seminumerical_dim_computation=True, timeout_fpoly=5, nbr_points=0)

### Decomposition  of  $\big\langle Δ_{123|4|5}, s_{123}-s_{3} \big\rangle$

In [56]:
I = Ideal(R5, [sympy.expand(4 * oPs("Δ_123|4|5")),
               sympy.expand(4 * oPs("(s_123-s_4)"))], )

In [57]:
J = Ideal(R5, [sympy.expand(4 * oPs("s_1234")),  # i.e. s_5
               sympy.expand(4 * oPs("(s_123-s_4)")),
              ], )

In [58]:
K = Ideal(R5, [sympy.expand(4 * oPs("(s_123-s_4)")),
               sympy.expand(4 * oPs("-tr(1+2+3+4|4)+4s_4")),  # i.e.: tr(5|4)+4s_4
              ], )

In [59]:
assert J.test_primality(verbose=False)

In [60]:
assert K.test_primality(verbose=False)

In [61]:
assert I == J & K

### Decomposition  of  $\big\langle ⟨1|3|2], Δ_{12|3|4|5} \big\rangle$

In [62]:
I = Ideal(R5, [sympy.expand(4 * oPs("⟨1|3|2]")), 
               sympy.expand(4 * oPs("Δ_12|3|4|5"))], )

In [63]:
J = Ideal(R5, [sympy.expand(4 * oPs("⟨1|3|2]")),
               sympy.expand(4 * oPs("⟨1|4|2]")),
               sympy.expand(4 * oPs("⟨1|3|2+3+4|1⟩")),  # ⟨1|3|5|1⟩
               sympy.expand(4 * oPs("[2|1+3+4|3|2]")),  # [2|5|3|2]
              ], )

In [64]:
K = Ideal(R5, [sympy.expand(4 * oPs("⟨1|3|2]"))] + 
          (numpy.vectorize(sympy.expand)(4 * oPs("|1|2|3|4|3|-|1|3|1|4|3|+|1|3|3|4|1+2|-|3|2|3|4|1+2|"))).flatten().tolist(),
         )

In [65]:
assert I == J & K

In [66]:
assert J.test_primality()

In [67]:
assert K.test_primality(verbose=False, seminumerical_dim_computation=True, timeout_fpoly=5, nbr_points=0)

Was irreducible at the chosen point. Continuing test.                    

In [68]:
# for example used in https://gdelaurentis.github.io/antares-results/antares_results/ttH/qqttH/pm/coeffs/tri12x3x00m.html